## Aprendizado Supervisionado
### prof. Duncan
Prática com Decision Tree Regressor e Diabetes

In [ ]:
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
#  algoritmo de aprendizado e métricas
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree, export_graphviz, export_text
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
#from sklearn.metrics import root_mean_squared_error as rmse
from sklearn.metrics import make_scorer
#
#    opções de seleção de modelo de entrada
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# pacote para amostragem
from sklearn.utils import resample

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv

#pacotes para visualização e formatação
import pprint
import graphviz

# configurações para os diferentes pacotes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# carga de dados
diabetes = datasets.load_diabetes(as_frame=True)
print(diabetes.DESCR)

In [ ]:
# separação em features e target
X = diabetes.data
y = diabetes.target
diabetes.frame

In [ ]:
# separação em treino e teste, e X e y

treino_X, teste_X, treino_y, teste_y = train_test_split(X, y, test_size=0.2, random_state=0)

print(treino_X.shape)
print(treino_y.shape)
print(teste_X.shape)
print(teste_y.shape)


## Experimento com reescala de valores, embaralhamento e busca exaustiva nos parâmetros

In [ ]:
# protocolo experimental com preparação e modelagem

# indução do modelo de classificação por árvore de decisão
ccp_alphas = [0.0, 0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0]
#ccp_alphas = [0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.04, 0.05, 0.055, 0.06, 0.07, 0.08, 0.09, 0.1]
#ccp_alphas = [0.055]
#ccp_alphas = [0.0]

mss=np.arange(132, 144, 1)  # default 2
msl=np.arange(38, 39, 1)   # default 1


modelo = Pipeline(steps=[
    ('reescala', MinMaxScaler()),
    ('modelagem', DecisionTreeRegressor())
    ])
parametros = {'modelagem__ccp_alpha':ccp_alphas,
              'modelagem__min_samples_split': mss,
              'modelagem__min_samples_leaf':msl  }
kfold = KFold(n_splits=10, shuffle=True, random_state=0)
grade = GridSearchCV(modelo, param_grid=parametros, cv=kfold, scoring=make_scorer(mse, greater_is_better=False), return_train_score=True)
grade.fit(treino_X,treino_y)
print('Melhor ccp_alpha:{}  score:{:0.3f}'.format(grade.best_params_, grade.best_score_))

In [ ]:
resultados = pd.DataFrame(grade.cv_results_, copy=True).sort_values(by='rank_test_score', ignore_index=True)
resultados.head()

In [ ]:
results = resultados[resultados['rank_test_score']==1].copy()

In [ ]:
results

In [ ]:
pd.to_numeric(results['param_modelagem__ccp_alpha']).describe()

## Execução para os melhores parâmetros

In [ ]:
estimador = DecisionTreeRegressor(ccp_alpha = resultados.param_modelagem__ccp_alpha[0],
                min_samples_split=resultados.param_modelagem__min_samples_split[0],
                min_samples_leaf=resultados.param_modelagem__min_samples_leaf[0]
)
estimador.fit(treino_X, treino_y)
teste_pred_y = estimador.predict(teste_X)
MAE = mae(teste_y, teste_pred_y)
RMSE = mse(teste_y, teste_pred_y, squared=True)
print('MAE=', MAE, '  RMSE=', RMSE)


In [ ]:
# apresentações do modelo
modelo_txt = export_text(estimador, feature_names=diabetes['feature_names'])
print(modelo_txt)

In [ ]:
# apresentações do modelo
plt.figure(figsize=(15, 15))
plot_tree(estimador, filled=True, rounded=True, feature_names=diabetes.feature_names)
plt.title('Árvore de Decisão treinada no dataset Diabetes')
plt.show()